In [1]:
GEOTIFF_PATH = "../dataset_marseille/satellite_raster_sample_laplaine_IGN_0.2m-resolution.tif"
BOXES_FILE = "../dataset_marseille/deepforest_pred_laplaine_boxes-IGN0.2m_patch400px_overlap0.15.csv"


In [2]:
import pandas as pd
import numpy as np
import rasterio
import geopandas
import shapely

In [3]:
%load_ext autoreload
%autoreload 1
%aimport functions

Using TensorFlow backend.


In [4]:
boxes = pd.read_csv(BOXES_FILE)

In [5]:
raster_path = GEOTIFF_PATH

with rasterio.open(raster_path) as dataset:
    print(dataset.crs)

EPSG:3857


In [6]:
# find center point for each box
boxes["center_x"] = boxes.apply(
     lambda b: b.xmin + (b.xmax-b.xmin)/2,
     axis=1)

boxes["center_y"] = boxes.apply(
    lambda b: b.ymin + (b.ymax-b.ymin)/2,
    axis=1)
    

boxes

,Unnamed: 0,xmin,ymin,xmax,ymax,score,label,center_x,center_y
0,0,2153,1284,2228,1358,0.889925,Tree,2190.5,1321.0
1,1,2605,1661,2659,1714,0.832605,Tree,2632.0,1687.5
2,2,2133,1601,2203,1665,0.829963,Tree,2168.0,1633.0
3,3,2628,1606,2681,1658,0.814309,Tree,2654.5,1632.0
4,4,2342,2061,2401,2125,0.811012,Tree,2371.5,2093.0
...,...,...,...,...,...,...,...,...,...
926,926,2279,3025,2319,3065,0.200908,Tree,2299.0,3045.0
927,927,2034,828,2050,845,0.200693,Tree,2042.0,836.5
928,928,2236,3036,2275,3078,0.200634,Tree,2255.5,3057.0
929,929,3005,3103,3021,3119,0.200237,Tree,3013.0,3111.0


In [7]:
raster_path = GEOTIFF_PATH

projection = functions.RasterProject(raster_path)

boxes["center_x"] = projection.projectX(boxes["center_x"])
boxes["center_y"] = projection.projectY(boxes["center_y"])

boxes = projection.project_boxes(boxes)

boxes

,Unnamed: 0,xmin,ymin,xmax,ymax,score,label,center_x,center_y
0,0,599568.545026,5.357036e+06,599583.544208,5.357021e+06,0.889925,Tree,599576.044617,5.357028e+06
1,1,599658.940097,5.356960e+06,599669.739509,5.356950e+06,0.832605,Tree,599664.339803,5.356955e+06
2,2,599564.545244,5.356972e+06,599578.544480,5.356960e+06,0.829963,Tree,599571.544862,5.356966e+06
3,3,599663.539847,5.356971e+06,599674.139269,5.356961e+06,0.814309,Tree,599668.839558,5.356966e+06
4,4,599606.342965,5.356880e+06,599618.142322,5.356868e+06,0.811012,Tree,599612.242643,5.356874e+06
...,...,...,...,...,...,...,...,...,...
926,926,599593.743652,5.356688e+06,599601.743216,5.356680e+06,0.200908,Tree,599597.743434,5.356684e+06
927,927,599544.746323,5.357127e+06,599547.946149,5.357124e+06,0.200693,Tree,599546.346236,5.357125e+06
928,928,599585.144121,5.356686e+06,599592.943695,5.356677e+06,0.200634,Tree,599589.043908,5.356681e+06
929,929,599738.935736,5.356672e+06,599742.135562,5.356669e+06,0.200237,Tree,599740.535649,5.356671e+06


In [12]:
# get the center of the box as a point

boxes['geometry'] = boxes.apply(
    lambda b: shapely.geometry.point.Point(b.center_x, b.center_y), 
    axis=1)
boxes = geopandas.GeoDataFrame(boxes, geometry='geometry')

boxes

,Unnamed: 0,xmin,ymin,xmax,ymax,score,label,center_x,center_y,geometry
0,0,599568.545026,5.357036e+06,599583.544208,5.357021e+06,0.889925,Tree,599576.044617,5.357028e+06,POINT (599576.045 5357028.453)
1,1,599658.940097,5.356960e+06,599669.739509,5.356950e+06,0.832605,Tree,599664.339803,5.356955e+06,POINT (599664.340 5356955.165)
2,2,599564.545244,5.356972e+06,599578.544480,5.356960e+06,0.829963,Tree,599571.544862,5.356966e+06,POINT (599571.545 5356966.063)
3,3,599663.539847,5.356971e+06,599674.139269,5.356961e+06,0.814309,Tree,599668.839558,5.356966e+06,POINT (599668.840 5356966.263)
4,4,599606.342965,5.356880e+06,599618.142322,5.356868e+06,0.811012,Tree,599612.242643,5.356874e+06,POINT (599612.243 5356874.077)
...,...,...,...,...,...,...,...,...,...,...
926,926,599593.743652,5.356688e+06,599601.743216,5.356680e+06,0.200908,Tree,599597.743434,5.356684e+06,POINT (599597.743 5356683.707)
927,927,599544.746323,5.357127e+06,599547.946149,5.357124e+06,0.200693,Tree,599546.346236,5.357125e+06,POINT (599546.346 5357125.338)
928,928,599585.144121,5.356686e+06,599592.943695,5.356677e+06,0.200634,Tree,599589.043908,5.356681e+06,POINT (599589.044 5356681.307)
929,929,599738.935736,5.356672e+06,599742.135562,5.356669e+06,0.200237,Tree,599740.535649,5.356671e+06,POINT (599740.536 5356670.509)


In [22]:
# # export test - to verify that the tree center points are correct in QGIS

# functions.save_shapefile('test_tree_center_points.shp', boxes)

In [26]:
import pyproj 

# transform from source data format (epsg:3857) to what the google street view api uses
transformer_to_gmaps = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326")

# to project the points back in qgis - where we have everything in epsg:3857
transformer_from_gmaps = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857")

In [87]:
# create strings of 100 lat lon to be used with google road api (to find the nearest road)

max_latlon = 99
queries = []
query = []
indices_to_box = {}
og_index_to_box = {}

i, j = 0,0
for index, box in enumerate(boxes[["xmin", "ymin", "center_x", "center_y", "geometry"]].values):
    xmin, ymin, center_x, center_y, geometry = box
    
    if index != 0 and index % max_latlon == 0:
        queries.append("|".join(query))
        query = []
        indices_to_box[i] = og_index_to_box
        og_index_to_box = {}
        i+=1
        j=0
 
    x = geometry.x
    y = geometry.y
    
    ## transform from one projection to what google street api uses
    x, y = transformer_to_gmaps.transform(x, y)

    query.append("{},{}".format(x, y))
    og_index_to_box[j] = index
    j+=1

print("total number of points: {}".format(len(query) + len(queries) * max_latlon))

# we're missing the last bits in query
queries.append("|".join(query))
indices_to_box[i] = og_index_to_box

# indices_to_box

total number of points: 931


In [117]:
# supply your own api key for google road api 
from google_api_key import api_key


In [119]:
def get_google_road_snappoint(query):
    url = "https://roads.googleapis.com/v1/nearestRoads?key={}&points={}".format(api_key, query)

    r = requests.get(url)
    api_response = r.json()
    return api_response

# get_google_road_snappoint(queries[0])

# check last one which sould be not full (less than 99)
# get_google_road_snappoint(queries[-1])

In [111]:
import requests

data = {}

google_road_data = []
for query_index, query in enumerate(queries):
    
    api_response = get_google_road_snappoint(query)
    
    points = api_response["snappedPoints"]
    
    for point in points:
        location = point["location"]
        x = location["latitude"]
        y = location["longitude"]
        placeId = point["placeId"]
        original_index = point["originalIndex"]
        box_index = indices_to_box[query_index][original_index]        
        l = data.get(box_index, [])
        l.append({"x_groad": x, "y_groad": y, "placeId": placeId})
        data[box_index] = l
        

# data

In [113]:
geometries = []

for box_index, points in data.items():
    box = boxes.loc[box_index]
    
    # XXX : hack only consider the first of possibly two points 
    # returned for closest road.
    
    points = [points[0]]
    for point in points:
        x = point['x_groad']
        y = point['y_groad']
        x, y = transformer_from_gmaps.transform(x, y) 
        geometries.append(shapely.geometry.LineString([(box.center_x, box.center_y), (x, y)]))
        
lines = pd.DataFrame(geometries, columns=['geometry'])
lines = geopandas.GeoDataFrame(lines, geometry='geometry')
lines

,geometry
0,"LINESTRING (599576.045 5357028.453, 599596.253..."
1,"LINESTRING (599664.340 5356955.165, 599701.897..."
2,"LINESTRING (599571.545 5356966.063, 599571.566..."
3,"LINESTRING (599668.840 5356966.263, 599701.897..."
4,"LINESTRING (599612.243 5356874.077, 599600.144..."
...,...
918,"LINESTRING (599597.743 5356683.707, 599565.034..."
919,"LINESTRING (599546.346 5357125.338, 599511.229..."
920,"LINESTRING (599589.044 5356681.307, 599565.129..."
921,"LINESTRING (599740.536 5356670.509, 599732.779..."


In [115]:
# export all the lines to the shapefile
functions.save_shapefile('tree_to_closest_roads.shp', lines)